In [11]:
import os
from openai import OpenAI, AzureOpenAI
from dotenv import load_dotenv
import json

In [12]:
load_dotenv()

api_key = os.environ.get("OPENAI_API_KEY")
azure_api_key = os.environ.get("AZURE_OPENAI_KEY")
azure_endpoint = os.environ.get("AZURE_OPENAI_ENDPOINT")

In [13]:
# Azure OpenAI API
# model = "gpt-35-turbo"
# client = AzureOpenAI(
#   azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
#   api_key=os.getenv("AZURE_OPENAI_KEY"),  
#   api_version="2024-02-15-preview"
# )

# OpenAI API
model = "gpt-3.5-turbo"
client = OpenAI(api_key=api_key)

### Example dummy function hard coded to return the same weather
#### In production, this could be your backend API or an external API

In [14]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps({"location": "San Francisco", "temperature": "72", "unit": unit})
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})


### Step 1: Send the conversation and available functions to GPT

In [15]:
messages = [{"role": "user", "content": "What's the weather like in San Francisco, Tokyo, and Paris?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }]

In [16]:
response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
    tools=tools,
    tool_choice="auto",  # auto is default, but we'll be explicit
)
response_message = response.choices[0].message
tool_calls = response_message.tool_calls



### Step 2: Check if the model wanted to call a function

In [20]:
print(tool_calls)

print()

for tool_call in tool_calls:
    print(tool_call)


[ChatCompletionMessageToolCall(id='call_cBEky3ggowc1XHPqgOj650J6', function=Function(arguments='{"location": "San Francisco"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_B7LENIlh6U87evv1GvIIpgmc', function=Function(arguments='{"location": "Tokyo"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_RmH0b1ivVuT7mvZtvppgIHlm', function=Function(arguments='{"location": "Paris"}', name='get_current_weather'), type='function')]

ChatCompletionMessageToolCall(id='call_cBEky3ggowc1XHPqgOj650J6', function=Function(arguments='{"location": "San Francisco"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_B7LENIlh6U87evv1GvIIpgmc', function=Function(arguments='{"location": "Tokyo"}', name='get_current_weather'), type='function')
ChatCompletionMessageToolCall(id='call_RmH0b1ivVuT7mvZtvppgIHlm', function=Function(arguments='{"location": "Paris"}', name='get_current_weather'), type='

### Step 3: Call the function

In [18]:
available_functions = {
    "get_current_weather": get_current_weather,
}  # only one function in this example, but you can have multiple
messages.append(response_message)  # extend conversation with assistant's reply
# Step 4: send the info for each function call and function response to the model
for tool_call in tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response

print(messages)
    


[{'role': 'user', 'content': "What's the weather like in San Francisco, Tokyo, and Paris?"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_cBEky3ggowc1XHPqgOj650J6', function=Function(arguments='{"location": "San Francisco"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_B7LENIlh6U87evv1GvIIpgmc', function=Function(arguments='{"location": "Tokyo"}', name='get_current_weather'), type='function'), ChatCompletionMessageToolCall(id='call_RmH0b1ivVuT7mvZtvppgIHlm', function=Function(arguments='{"location": "Paris"}', name='get_current_weather'), type='function')]), {'tool_call_id': 'call_cBEky3ggowc1XHPqgOj650J6', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "San Francisco", "temperature": "72", "unit": null}'}, {'tool_call_id': 'call_B7LENIlh6U87evv1GvIIpgmc', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "Tokyo", "temp

In [19]:
second_response = client.chat.completions.create(
    model="gpt-3.5-turbo-0125",
    messages=messages,
)  # get a new response from the model where it can see the function response
print(second_response.choices[0].message.content)

The current weather in:

- San Francisco: 72°F
- Tokyo: 10°C
- Paris: 22°C
